In [ ]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.svm import SVR,SVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB , GaussianNB
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
classification_models = {
                           "LogisticRegression" : {
                                                        'model' : LogisticRegression(),
                                                        'para' : {
                                                                      'penalty': ['l1','l2'],
                                                                      'C' : [0.001,0.1,1,5,10]
                                                        }
                           },
                            "RandomForestClassifier" : {
                                                        'model' : RandomForestClassifier(),
                                                        'para' : {
                                                                'n_estimators': [200,300],
                                                                'max_depth': [10, 20, None],
                                                                'min_samples_leaf': [1, 5, 10],
                                                                'max_features': ['sqrt', 'log2']
                                                                }
                             },
                             "KNN" : {
                                        'model' : KNeighborsClassifier(),
                                        'para' : {
                                                    'n_neighbors' : range(1,11)
                                                 }
                            },
                             "SVC" : {
                                        'model' : SVC(),
                                        'para' : {
                                                    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                                                    'C' : [0.001,0.1,1,5,10],
                                                    'gamma' : ['scale', 'auto']
                                                 }
                            },
                            "BernoulliNB" : {
                                        'model' : BernoulliNB(),
                                        'para' : {
                                                    'alpha': [0.01,0.1,10]
                                                 }
                            },
                            "GaussianNB" : {
                                        'model' : GaussianNB(),
                                        'para' : {
                                                    'var_smoothing' : [1e-8,1e-9,1e-10,1e-11]
                                                 }
                            },
}

In [ ]:
regression_models = {
                           "LinearRegression" : {
                                                        'model' : LinearRegression(),
                                                        'para' : {
                                                                      'fit_intercept': [True,False]
                                                        }
                           },
                           "Lasso" : {
                                                        'model' : Lasso(),
                                                        'para' : {
                                                                  'alpha' : [0.001,0.01,0.1,1,10],
                                                                  'max_iter':  [200,500,1000]  
                                                        }
                           },
                           "Ridge" : {
                                                        'model' : Ridge(),
                                                        'para' : {
                                                                  'alpha' : [0.001,0.01,0.1,1,10],
                                                                  'max_iter':  [200,500,1000]  
                                                        }
                           },
                            "RandomForestRegressor" : {
                                                        'model' : RandomForestRegressor(),
                                                        'para' : {
                                                                'n_estimators': [200,300],
                                                                'max_depth': [10, 20, None],
                                                                'min_samples_leaf': [1, 5, 10],
                                                                'max_features': ['sqrt', 'log2']
                                                                }
                             },
                             "KNN" : {
                                        'model' : KNeighborsRegressor(),
                                        'para' : {
                                                    'n_neighbors' : range(1,11)
                                                 }
                            },
                             "SVR" : {
                                        'model' : SVR(),
                                        'para' : {
                                                    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                                                    'C' : [0.001,0.1,1,5,10],
                                                    'gamma' : ['scale', 'auto']
                                                 }
                           },
}

In [ ]:
def clean(x,target):
    x = x.dropna(axis = 1,how = 'all')
    x = x.dropna(subset = [target])
    imputer = SimpleImputer(strategy='most_frequent')
    imputed_data = imputer.fit_transform(x)
    #print(imputed_data.dtype.names)
    return pd.DataFrame(imputed_data,columns= x.columns)

def feature_engineering(x):
    x = x.apply(pd.to_numeric, errors='ignore')
    cat_features = x.select_dtypes(include='object')
    num_features = x.select_dtypes(exclude='object')
    print(cat_features.columns)
    for col in cat_features.columns:
        if len(cat_features[col].unique()) > 5:
            x = x.drop(col,axis = 1)
            print(col)

    for col in num_features.columns:
        varience = np.var(num_features[col])
        print(f'{col} : {varience}')
        if varience > -0.5 and varience < .5:
            x = x.drop(col,axis = 1)
            print(col)

    return x

def outlier_detection(X,target):
    num = X.select_dtypes(include='number').columns
    for col in num:
        upper_limit = X[col].mean() + 3*X[col].std()
        lower_limit = X[col].mean() - 3*X[col].std()
        X = X[(X[col] > lower_limit) & (X[col] < lower_limit)]
    return X

def X_encoding(x):
    ohe = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
    cat_features = x.select_dtypes(include='object')
    encoded = ohe.fit_transform(cat_features)
    result = x.drop(cat_features,axis = 1)
    result = pd.concat([result,encoded],axis=1)
    return result

def y_encoding(y):
    try:
        y = y.apply(pd.to_numeric)
        print('done',y.iloc[0].dtype)
        return y
    except:
        uni = y.iloc[0].unique()
        num = len(uni)
        for i in range(num):
            y.iloc[0] = y.iloc[0].apply(lambda x: i if x == uni[i] else x)
        return y

def iscat(x):
    return True if x.iloc[0].dtype == 'object'  else False

def data_scale(x):
    x = x.apply(pd.to_numeric, errors='ignore')
    numeric_col = x.select_dtypes(include='number')
    numeric_feature_name = numeric_col.columns
    scale = StandardScaler()
    x[numeric_feature_name] = scale.fit_transform(x[numeric_feature_name])
    return x

def dimention_reduction(x):
    pca = PCA(0.95).set_output(transform='pandas')
    x = pca.fit_transform(x)
    return x

In [ ]:
data = pd.read_csv("D:\\DataSet\\Titanic\\archive\\Titanic-Dataset.csv")
data.info()

In [ ]:
data = clean(data,'Survived')
data.isna().sum()

In [ ]:
data = outlier_detection(data,'Survived')

In [ ]:
X_axis = data.drop(['Survived'],axis=1)
X_axis.head()

In [ ]:
X_axis = feature_engineering(X_axis)
X_axis.head()

In [ ]:
X_axis = X_encoding(X_axis)
X_axis

In [ ]:
X_axis = data_scale(X_axis)

In [ ]:
#X_axis = dimention_reduction(X_axis)

In [ ]:
y_axis = data[['Survived']]

y_axis = y_encoding(y_axis)

In [ ]:
X_axis.head()

In [ ]:
#y_axis = pd.to_numeric(y_axis['Survived'])

In [ ]:
for model_name,model in classification_models.items():
    grid = GridSearchCV(model['model'],model['para'],cv=3,n_jobs=-1)
    grid.fit(X_axis,y_axis.values.ravel())
    print(f'model : {model_name}')
    print(f'Best Score : {grid.best_score_}')
    print(f'Best parameter : {grid.best_params_}')

In [ ]:
def reg_clean(x,target):
    x = x.dropna(how = all)
    x = x.dropna([target])
    
    return x

In [ ]:
reg_data = pd.read_csv("C:\\Users\\Debjit\\Downloads\\Housing.csv")
reg_data = pd.DataFrame(reg_data)
reg_data.head()

In [ ]:

#reg_data = reg_data.dropna(axis=1, how='all')

In [ ]:
reg_data = clean(reg_data,'price')
reg_data.head()

In [ ]:
reg_data = outlier_detection(reg_data,'Price')

In [ ]:
reg_X_axis = reg_data.drop(['price'],axis = 1)
reg_y_axis = reg_data[['price']]

In [ ]:
reg_X_axis.head()

In [ ]:
reg_y_axis.head()

In [ ]:
reg_X_axis = feature_engineering(reg_X_axis)
reg_X_axis.head()


reg_X_axis.info()

In [ ]:
#reg_X_axis = reg_X_axis.drop(['Id'],axis=1)
reg_X_axis = X_encoding(reg_X_axis)
reg_X_axis.head()

In [ ]:
reg_X_axis = data_scale(reg_X_axis)
reg_X_axis.head()

In [ ]:
reg_y_axis.info()

In [ ]:
reg_y_axis = y_encoding(reg_y_axis)
reg_y_axis.head()

In [ ]:
for model_name,model in regression_models.items():
    grid = GridSearchCV(model['model'],model['para'],cv=3,n_jobs=-1,scoring='r2')
    grid.fit(reg_X_axis,reg_y_axis.values.ravel())
    print(f'model : {model_name}')
    print(f'Best Score : {grid.best_score_}')
    print(f'Best parameter : {grid.best_params_}')

In [ ]:
churn = pd.read_csv("C:\\Users\\Debjit\\Downloads\\telecom_churn.csv")
churn.head()

In [ ]:
churn = clean(churn,'Churn')
churn.info()

In [ ]:
#churn = outlier_detection(churn,'Churn')
#churn.info()

In [ ]:
churn_X = churn.drop(['Churn'],axis=1)
churn_y = churn[['Churn']]

In [ ]:
churn_y.head()

In [ ]:
churn_X = feature_engineering(churn_X)

In [ ]:
churn_X = data_scale(churn_X)

In [ ]:
if iscat(churn_y):
    churn_y = y_encoding(churn_y)

In [ ]:
for model_name,model in classification_models.items():
    grid = GridSearchCV(model['model'],model['para'],cv=3,n_jobs=-1)
    grid.fit(churn_X,churn_y.values.ravel())
    print(f'model : {model_name}')
    print(f'Best Score : {grid.best_score_}')
    print(f'Best parameter : {grid.best_params_}')

In [249]:
student_data = pd.read_csv("C:\\Users\\Debjit\\Downloads\\student_habits_performance.csv")

student_data.drop(['student_id'],axis = 1,inplace=True)

student_data.head()

,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
0,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2
1,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0
2,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3
3,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8
4,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4


In [250]:
student_data = clean(student_data,'exam_score')
student_data.isna().sum()

age                              0
gender                           0
study_hours_per_day              0
social_media_hours               0
netflix_hours                    0
part_time_job                    0
attendance_percentage            0
sleep_hours                      0
diet_quality                     0
exercise_frequency               0
parental_education_level         0
internet_quality                 0
mental_health_rating             0
extracurricular_participation    0
exam_score                       0
dtype: int64

In [251]:
student_X = student_data.drop(['exam_score'],axis =1)
student_y = student_data['exam_score']

In [252]:
student_X = feature_engineering(student_X)
student_X.head()

Index(['gender', 'part_time_job', 'diet_quality', 'parental_education_level',
       'internet_quality', 'extracurricular_participation'],
      dtype='object')
age : 5.3219959999999995
study_hours_per_day : 2.15547999
social_media_hours : 1.3731997499999997
netflix_hours : 1.15472191
attendance_percentage : 88.25748511
sleep_hours : 1.50249599
exercise_frequency : 4.098236
mental_health_rating : 8.100156


C:\Users\Debjit\AppData\Local\Temp\ipykernel_19512\2278104783.py:10: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  x = x.apply(pd.to_numeric, errors='ignore')


,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation
0,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes
1,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No
2,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No
3,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes
4,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No


In [253]:
student_X = X_encoding(student_X)
student_X.head()

,age,study_hours_per_day,social_media_hours,netflix_hours,attendance_percentage,sleep_hours,exercise_frequency,mental_health_rating,gender_Female,gender_Male,...,diet_quality_Good,diet_quality_Poor,parental_education_level_Bachelor,parental_education_level_High School,parental_education_level_Master,internet_quality_Average,internet_quality_Good,internet_quality_Poor,extracurricular_participation_No,extracurricular_participation_Yes
0,23,0.0,1.2,1.1,85.0,8.0,6,8,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,20,6.9,2.8,2.3,97.3,4.6,6,8,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,21,1.4,3.1,1.3,94.8,8.0,1,1,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3,23,1.0,3.9,1.0,71.0,9.2,4,1,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,19,5.0,4.4,0.5,90.9,4.9,3,1,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [254]:
student_X = data_scale(student_X)
student_y = y_encoding(student_y)

done float64


C:\Users\Debjit\AppData\Local\Temp\ipykernel_19512\2278104783.py:60: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  x = x.apply(pd.to_numeric, errors='ignore')


In [255]:
for model_name,model in regression_models.items():
    grid = GridSearchCV(model['model'],model['para'],cv=3,n_jobs=-1)
    grid.fit(student_X,student_y.values.ravel())
    print(f'model : {model_name}')
    print(f'Best Score : {grid.best_score_}')
    print(f'Best parameter : {grid.best_params_}')

model : LinearRegression
Best Score : 0.8978575563317291
Best parameter : {'fit_intercept': True}
model : Lasso
Best Score : 0.8986537430676891
Best parameter : {'alpha': 0.1, 'max_iter': 200}
model : Ridge
Best Score : 0.8978739609635138
Best parameter : {'alpha': 1, 'max_iter': 200}
model : RandomForestRegressor
Best Score : 0.7271872280124049
Best parameter : {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'n_estimators': 300}
model : KNN
Best Score : 0.41890096238084945
Best parameter : {'n_neighbors': 9}
model : SVR
Best Score : 0.8950810999890759
Best parameter : {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


In [294]:
wine_data = pd.read_csv("C:\\Users\\Debjit\\Downloads\\WineQT.csv")
wine_data.drop(['Id'],axis = 1,inplace=True)
wine_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [295]:
wine_data = clean(wine_data,'quality')
#wine_data.isna().sum()

In [296]:
wine_data_X = wine_data.drop(['quality'],axis =1)
wine_data_y = wine_data['quality']

In [297]:
wine_data_X = feature_engineering(wine_data_X)
wine_data_X.head()

Index([], dtype='object')
fixed acidity : 3.0514163507339362
volatile acidity : 0.032239852990817094
volatile acidity
citric acid : 0.03865147908567423
citric acid
residual sugar : 1.8369036793629143
chlorides : 0.002232246555357308
chlorides
free sulfur dioxide : 104.98053884996659
total sulfur dioxide : 1073.7278504557012
density : 3.7026412132429187e-06
density
pH : 0.024522154634432723
pH
sulphates : 0.029010318810761083
sulphates
alcohol : 1.1701227121932987


C:\Users\Debjit\AppData\Local\Temp\ipykernel_19512\2278104783.py:10: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  x = x.apply(pd.to_numeric, errors='ignore')


,fixed acidity,residual sugar,free sulfur dioxide,total sulfur dioxide,alcohol
0,7.4,1.9,11.0,34.0,9.4
1,7.8,2.6,25.0,67.0,9.8
2,7.8,2.3,15.0,54.0,9.8
3,11.2,1.9,17.0,60.0,9.8
4,7.4,1.9,11.0,34.0,9.4


In [298]:
wine_data_X = X_encoding(wine_data_X)
wine_data_X.head()

,fixed acidity,residual sugar,free sulfur dioxide,total sulfur dioxide,alcohol
0,7.4,1.9,11.0,34.0,9.4
1,7.8,2.6,25.0,67.0,9.8
2,7.8,2.3,15.0,54.0,9.8
3,11.2,1.9,17.0,60.0,9.8
4,7.4,1.9,11.0,34.0,9.4


In [299]:
wine_data_X = data_scale(wine_data_X)
wine_data_y = y_encoding(wine_data_y)

done float64


C:\Users\Debjit\AppData\Local\Temp\ipykernel_19512\2278104783.py:60: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  x = x.apply(pd.to_numeric, errors='ignore')


In [300]:
for model_name,model in regression_models.items():
    grid = GridSearchCV(model['model'],model['para'],cv=3,n_jobs=-1)
    grid.fit(wine_data_X,wine_data_y.values.ravel())
    print(f'model : {model_name}')
    print(f'Best Score : {grid.best_score_}')
    print(f'Best parameter : {grid.best_params_}')

model : LinearRegression
Best Score : 0.23316269012614185
Best parameter : {'fit_intercept': True}
model : Lasso
Best Score : 0.2341827189580258
Best parameter : {'alpha': 0.01, 'max_iter': 200}
model : Ridge
Best Score : 0.23372063855617006
Best parameter : {'alpha': 10, 'max_iter': 200}
model : RandomForestRegressor
Best Score : 0.232647171318751
Best parameter : {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 10, 'n_estimators': 200}
model : KNN
Best Score : 0.19715072927620755
Best parameter : {'n_neighbors': 10}
model : SVR
Best Score : 0.2344712196642916
Best parameter : {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}


In [305]:
vehical_data = pd.read_csv("C:\\Users\\Debjit\\Downloads\\CAR DETAILS FROM CAR DEKHO.csv")
vehical_data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [308]:
vehical_data = clean(vehical_data,'selling_price')
vehical_data_X = vehical_data.drop(['selling_price'],axis =1)
vehical_data_y = vehical_data['selling_price']

In [309]:
vehical_data_X = feature_engineering(vehical_data_X)
vehical_data_X.head()

Index(['name', 'fuel', 'seller_type', 'transmission', 'owner'], dtype='object')
name
year : 17.765030261844593
km_driven : 2175170962.4744215


C:\Users\Debjit\AppData\Local\Temp\ipykernel_19512\2278104783.py:10: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  x = x.apply(pd.to_numeric, errors='ignore')


,year,km_driven,fuel,seller_type,transmission,owner
0,2007,70000,Petrol,Individual,Manual,First Owner
1,2007,50000,Petrol,Individual,Manual,First Owner
2,2012,100000,Diesel,Individual,Manual,First Owner
3,2017,46000,Petrol,Individual,Manual,First Owner
4,2014,141000,Diesel,Individual,Manual,Second Owner


In [310]:
vehical_data_X = X_encoding(vehical_data_X)
vehical_data_X.head()

,year,km_driven,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2007,70000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,2007,50000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,2012,100000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,2017,46000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,2014,141000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [312]:
vehical_data_X = data_scale(vehical_data_X)
vehical_data_X.head()

C:\Users\Debjit\AppData\Local\Temp\ipykernel_19512\2278104783.py:60: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  x = x.apply(pd.to_numeric, errors='ignore')


,year,km_driven,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,-1.445074,0.081139,-0.096449,-0.992196,-0.015181,-0.072992,1.021899,-0.545042,0.581253,-0.155139,-0.339276,0.339276,0.729716,-0.137908,-0.584800,-0.062709,-0.274449
1,-1.445074,-0.347689,-0.096449,-0.992196,-0.015181,-0.072992,1.021899,-0.545042,0.581253,-0.155139,-0.339276,0.339276,0.729716,-0.137908,-0.584800,-0.062709,-0.274449
2,-0.258795,0.724381,-0.096449,1.007865,-0.015181,-0.072992,-0.978571,-0.545042,0.581253,-0.155139,-0.339276,0.339276,0.729716,-0.137908,-0.584800,-0.062709,-0.274449
3,0.927485,-0.433455,-0.096449,-0.992196,-0.015181,-0.072992,1.021899,-0.545042,0.581253,-0.155139,-0.339276,0.339276,0.729716,-0.137908,-0.584800,-0.062709,-0.274449
4,0.215717,1.603479,-0.096449,1.007865,-0.015181,-0.072992,-0.978571,-0.545042,0.581253,-0.155139,-0.339276,0.339276,-1.370396,-0.137908,1.709986,-0.062709,-0.274449


In [313]:
vehical_data_y = y_encoding(vehical_data_y)

done int64


In [314]:
for model_name,model in regression_models.items():
    grid = GridSearchCV(model['model'],model['para'],cv=3,n_jobs=-1)
    grid.fit(vehical_data_X,vehical_data_y.values.ravel())
    print(f'model : {model_name}')
    print(f'Best Score : {grid.best_score_}')
    print(f'Best parameter : {grid.best_params_}')

model : LinearRegression
Best Score : 0.4327655236654249
Best parameter : {'fit_intercept': True}


c:\Users\Debjit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.799e+12, tolerance: 1.452e+11
  model = cd_fast.enet_coordinate_descent(


model : Lasso
Best Score : 0.43297984463985423
Best parameter : {'alpha': 10, 'max_iter': 200}
model : Ridge
Best Score : 0.43295454181828247
Best parameter : {'alpha': 10, 'max_iter': 200}
model : RandomForestRegressor
Best Score : 0.6599805661590804
Best parameter : {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 200}
model : KNN
Best Score : 0.6121395018192352
Best parameter : {'n_neighbors': 8}
model : SVR
Best Score : 0.02155371050173675
Best parameter : {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
